# Generate Polar Model

Change the importing of models and lists acording to which model you want to generate.

### Import Packages

In [1]:
import gensim
from numpy import linalg
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os
import pandas as pd
import random
import scipy
import torch
import subprocess

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath

### Import Model

In [2]:
model_gn = gensim.models.KeyedVectors.load_word2vec_format('/Users/stjepankusenic/POLAR_WEBE/data/raw/glove_norm_300.mod',binary=True)
current_model = model_gn

### Import Polar Dimension List

In [3]:
#load original antonyms
list_antonym = pd.read_pickle(r'/Users/stjepankusenic/POLAR_WEBE/data/interim/final_antonym_list')

In [ ]:
#load business antonym list
list_new= [('product', 'service'),
('essential', 'luxury'),
('technical', 'natural'),
('renewable', 'nonrenewable'),
('advertising', 'secretive'),
('lease', 'sell'),
('tangible', 'intangible'),
('demand', 'supply'),
#('wfh', 'wfo'),
('child', 'childless'),
('remote', 'physical'),
('salary', 'goodies'),
('store', 'online'),
('details', 'outlines'),
('stakeholders', 'spectators'),
('isolating', 'social'),
('goal', 'task'),
('employees', 'consultant'),
('cost', 'revenue'),
('seasonal', 'temporary'),
('alliance', 'proprietorship'),
('loss', 'profit'),
('integrity', 'corruption'),
('international', 'local'),
('corporate', 'individual'),
('order', 'disorder'),
('solution', 'problem'),
('manager', 'worker'),
('diversity', 'uniformity'),
('public', 'private'),
('strategic', 'impulsive'),
('innovator', 'follower'),
('bankruptcy', 'prosperity'),
('growth', 'decline'),
('sustainable', 'unsustainable'),
('family', 'work'),
('criminal', 'rightful'),
('financial', 'artisanal'),
('supplier', 'purchaser'),
('commitment', 'rejection'),
('professional', 'amateur'),
('independent', 'dependent'),
('digital', 'analogue'),
('marketing', 'secret'),
('secure', 'risky'),
('longterm', 'shortterm'),
('responsible', 'neglect'),
('ethical', 'unethical'),
('beneficial', 'harmful'),
('diversity', 'uniformity'),
('trust', 'mistrust'),
('teamwork', 'individualism'),
('opportunity', 'threat'),
('innovative', 'traditional'),
('flexible', 'rigid'),
('ambiguity', 'clarity'),
('feminine', 'masculine'),
('globally', 'locally'),
('insiders', 'outsiders'),
('foreigners', 'natives'),
('minorities', 'majority'),
('transparency', 'obscurity'),
('discrimination', 'impartial'),
('credible', 'deceptive'),
('environment', 'pollution'),
('pressure', 'relax'),
('growth', 'decline'),
('satisfied', 'unsatisfied'),
('diplomatic', 'undiplomatic'),
('motivate', 'demotivate'),
('communicative', 'uncommunicative'),
('connected', 'disconnected'),
('autonomous', 'micromanagement'),
('nurture', 'neglect'),
('progressive', 'conservative'),
('rewarding', 'unrewarding'),
('bias', 'unbias'),
('challenge', 'obscurity'),
('collaboration', 'silo'),
('outdated', 'modern'),
('effortless', 'demanding'),
('economic', 'overpriced'),
('widespread', 'local'),
('freedom', 'captive'),
('consistent', 'inconsistent')]

list_new= list(dict.fromkeys(list_new).keys())

similarity_matrix = defaultdict(list)
for each_pair in tqdm(list_new):
    word1 = each_pair[0]
    word2 = each_pair[1]
    if word1 < word2:
        similarity_matrix[word1].append(word2)
    else:
        similarity_matrix[word2].append(word1)

all_similarity = defaultdict(dict)
for each_key in tqdm(similarity_matrix):
    for each_value in similarity_matrix[each_key]:
#         cosine_similarity([current_model[each_key]]
        all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])

final_list = []
for index_counter, each_key in enumerate(tqdm(all_similarity)):
#     print(each_key,all_similarity[each_key])
    listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
#     print(listofTuples)
    final_list.append((each_key, listofTuples[0][0]))
print(len(final_list))

list_antonym = final_list

In [ ]:
#display(list_antonym)
type(current_model[list_antonym[0][0]])

### Decide Polar Dimesion Size

In [4]:
num_antonym = 500

## Find the antonym difference vectors
antonymy_vector = []
for each_word_pair in list_antonym:
    if each_word_pair[0] in current_model.vocab:
        if each_word_pair[1] in current_model.vocab:
            antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
antonymy_vector = np.array(antonymy_vector)
print(antonymy_vector.shape)

(1468, 300)


### Implement Dimesion Selection Method

In [5]:
random.seed(42)

t1 = np.array(antonymy_vector)
dimension_similarity_matrix = scipy.spatial.distance.cdist(np.array(antonymy_vector),np.array(antonymy_vector),'cosine')
dimension_similarity_matrix = abs(1-dimension_similarity_matrix)

def get_set_score(final_list, each_dim):
    final_output = 0.0
    for each_vec in final_list:
        final_output += dimension_similarity_matrix[each_vec][each_dim]
    return final_output/(len(final_list))

def select_subset_dimension(dim_vector, num_dim):
    working_list = np.array(dim_vector)

    working_position_index = [i for i in range(working_list.shape[0])]
    final_position_index = []


    print('working list is ready, shape', working_list.shape)
    sel_dim = random.randrange(0, working_list.shape[0])

    final_position_index.append(sel_dim)

    working_position_index.remove(sel_dim)

    for test_count in tqdm(range(num_dim-1)):
        min_dim = None
        min_score = 1000
        for temp_index, each_dim in enumerate(working_position_index):
            temp_score = get_set_score(final_position_index, each_dim)
            if temp_score< min_score:
                min_score= temp_score
                min_dim = each_dim
        final_position_index.append(min_dim)
        working_position_index.remove(min_dim)
    return final_position_index

In [6]:
embedding_size = antonymy_vector.shape[0]
print('The embedding size is', embedding_size)


variance_antonymy_vector_inverse = np.linalg.pinv(np.transpose(antonymy_vector))
variance_antonymy_vector_inverse = torch.tensor(variance_antonymy_vector_inverse)

embedding_matrix = []

current_model_tensor = torch.t(torch.tensor(current_model.wv.vectors))

The embedding size is 1468


/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_42368/1149897526.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  current_model_tensor = torch.t(torch.tensor(current_model.wv.vectors))


In [7]:
var_list = [None for x in range(20)] # variance for each antonym in each batch

for i in range(19):  # the first 19 batches, each of size 100k
  temp = torch.matmul(variance_antonymy_vector_inverse, current_model_tensor[:,100000*i:100000*i+100000])
  temp_var_mean = torch.var(temp, axis = 1)
  var_list[i] = temp_var_mean.numpy()
  del temp

temp = torch.matmul(variance_antonymy_vector_inverse, current_model_tensor[:,1900000:])
temp_var_mean = torch.var(temp, axis = 1)
var_list[19] = temp_var_mean.numpy()
del temp

In [8]:
# using lazy approach. assume each batch is independent and the overall variance is the average variance over all batches

variance_list = np.mean(np.array(var_list),axis = 0)

variance_antonymy_vector = [each for each in sorted(range(len(variance_list)), key=lambda i: variance_list[i], reverse=True)]

### Import business entity names

In [9]:
company = pd.read_csv('/Users/stjepankusenic/POLAR_WEBE/data/raw/International_Fortune_GloVe.csv')
name_list = company['0']

In [10]:
name_word_embedding = dict()
for name in name_list:
    name_word_embedding[name] = current_model[name]

### Create Polar Embeddings

In [11]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):

    temp_dict = dict()

    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    print('New model size is',len(current_model), embedding_size)

    temp_file = None

    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)
        temp_dict[each_word] = new_vector

        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()
    return temp_dict

In [12]:
def generate_embedding_path(current_model, embedding_path, binary, antonym_vector, curr_dim):
    curr_antonym_vector = antonymy_vector[antonym_vector[:curr_dim]]
    curr_antonymy_vector_inverse = np.linalg.pinv(np.transpose(curr_antonym_vector))
    new_embedding_dict = transform_to_antonym_space(current_model, embedding_path, binary,curr_antonymy_vector_inverse)

    return new_embedding_dict

In [13]:
dim_size = 500 # Number of POLAR dimenions
antonym_vector_method = variance_antonymy_vector
# random_antonym_vector or orthogonal_antonymy_vector or variance_antonymy_vector
antonym_500 = [list_antonym[x] for x in antonym_vector_method[:500]]

In [14]:
# create POLAR embedding of names
name_new_embedding = generate_embedding_path(name_word_embedding, 'name_embeddings',True ,antonym_vector_method,500)

New model size is 167 500


/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_42368/2385129348.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for each_word in tqdm(current_model):


  0%|          | 0/167 [00:00<?, ?it/s]

In [15]:
def make_polar_dict(company_name, antonym, embedding, top_n = False, n = 10):
  temp_dict = dict()
  temp_polar = embedding[company_name]

  if top_n:
    idx = np.argsort([abs(x) for x in temp_polar])[-n:]
    for i in idx:
      print(antonym[i],temp_polar[i],'\n')


  if len(antonym) == len(temp_polar):
    for a in range(len(antonym)):
      temp_dict[antonym[a]] = temp_polar[a]
    return temp_dict

In [16]:
facebook_polar = make_polar_dict('facebook', antonym_500, name_new_embedding, True)

('follower', 'innovator') 0.09955804 

('critique', 'guess') 0.106556386 

('bragging', 'humble') 0.108869635 

('sign', 'talk') 0.11223837 

('comply', 'helm') 0.12356726 

('chapter', 'page') -0.12495641 

('supervised', 'unsupervised') -0.12506409 

('australian', 'kiwi') -0.12907332 

('disclose', 'secrete') 0.13103342 

('ceo', 'workforce') 0.14726065 



### Save the Polar Model

In [17]:
###make csv###
# download csv

df = dict()
for t in name_list:
  df[t] = make_polar_dict(t, antonym_500, name_new_embedding)

new_df = pd.DataFrame(df).transpose()

# change columns to better read names
new_columns = []

for pair in antonym_500:
  temp = pair[0]+''+pair[1]
  new_columns.append(temp)

new_df.columns = new_columns

In [18]:
new_df.to_csv('POLAR-GloVeWiki-org-antonyms-inter.csv')

#### Next up generate insights with the model